# Build Pipeline Notebook

This notebook will exercise the drift detection MLOps `build pipeline`

In [ ]:
%%capture
!pip install -U pandas seaborn

## Setup

👇 Set the project name for your drift pipeline and store variable

In [ ]:
project_name = "<<project_name>>"  # << Update this drift detection project
%store project_name

Get back the project id and region

In [ ]:
import sagemaker
import json

sess = sagemaker.session.Session()
region_name = sess._region_name
sm_client = sess.sagemaker_client
project_id = sm_client.describe_project(ProjectName=project_name)["ProjectId"]
artifact_bucket = f"sagemaker-project-{project_id}-{region_name}"

print(f"Project: {project_name} ({project_id})")

## Data Prep

Let's copy some trip data and taxi zone files to the input location

In [ ]:
from sagemaker.s3 import S3Downloader, S3Uploader

# Download trip data and taxi zones to input folder
download_uri = "s3://nyc-tlc/trip data/green_tripdata_2018-02.csv"
S3Downloader().download(download_uri, "input/data")
download_uri = "s3://nyc-tlc/misc/taxi_zones.zip"
S3Downloader().download(download_uri, "input/zones")

# Upload input to the target location
input_data_uri = f"s3://{artifact_bucket}/{project_id}/input"
S3Uploader().upload("input", input_data_uri)

print("Listing input files:")
for s3_uri in S3Downloader.list(input_data_uri):
    print(s3_uri)

## Train

Start the pipeline now that we have uploaded some data

In [ ]:
from sagemaker.workflow.pipeline import Pipeline

pipeline_name = f"{project_name}-build"
pipeline = Pipeline(pipeline_name)

# Start pipeline
execution = pipeline.start()
execution_name = execution.arn.split("/")[-1]

print(f"Waiting for execution: {execution_name} for pipeline {pipeline_name}...")
execution.wait()
execution_status = execution.describe()["PipelineExecutionStatus"]
print(f"Status: {execution_status}")

List the execution steps.  Note that we have baseline and training jobs.

In [ ]:
for step in execution.list_steps():
    print("Step: {}, Status: {}".format(step["StepName"], step["StepStatus"]))

### Evaluate

Get the estimator for the training job in the pipeline.

In [ ]:
from sagemaker.estimator import Estimator


def get_execution_step(step_name):
    return [
        step["Metadata"]
        for step in execution.list_steps()
        if step["StepName"] == step_name
    ]


training_job_arn = get_execution_step("TrainModel")[0]["TrainingJob"]["Arn"]
training_job_name = training_job_arn.split("/")[-1]
estimator = Estimator.attach(training_job_name)

### Download the Debugger XGBoost training report

SageMaker Debugger generates a [XGBoost Training Report](https://docs.aws.amazon.com/sagemaker/latest/dg/debugger-training-xgboost-report.html) by a processing jobs that run concurrent to the training job. Let's wait for it to complete.

In [ ]:
# get name of the xgboost training report
xgb_report_job_name = [
    rule["RuleEvaluationJobArn"].split("/")[-1]
    for rule in estimator.latest_training_job.rule_job_summary()
    if "CreateXgboostReport" in rule["RuleConfigurationName"]
][0]

print("Waiting for XGBoost training report to complete...")
sm_client.get_waiter("processing_job_completed_or_stopped").wait(
    ProcessingJobName=xgb_report_job_name
)
print("Done")

ℹ️ The code below will download the output from the Debugger report in the `report` folder.  Click the link to open the report.

In [ ]:
from IPython.display import FileLink
from sagemaker.s3 import S3Downloader, S3Uploader

# Get the s3 output
report_uri = sm_client.describe_processing_job(ProcessingJobName=xgb_report_job_name)[
    "ProcessingOutputConfig"
]["Outputs"][0]["S3Output"]["S3Uri"]

# Download the notebook from the report
S3Downloader().download(f"{report_uri}/xgboost_report.html", "report")
FileLink("report/xgboost_report.html", result_html_prefix="Open Report: ")

### Approve Model

ℹ️ Once we are happy with this training job, we can [Update the Approval Status](https://docs.aws.amazon.com/sagemaker/latest/dg/model-registry-approve.html) of a model.

In [ ]:
model_package_arn = get_execution_step("RegisterModel")[0]["RegisterModel"]["Arn"]
model_package_version = model_package_arn.split("/")[-1]
print(f"Model version: {model_package_version}")

Let's update the status to approved

In [ ]:
model_package_update_input_dict = {
    "ModelPackageArn": model_package_arn,
    "ModelApprovalStatus": "Approved",
}
model_package_update_response = sm_client.update_model_package(
    **model_package_update_input_dict
)

## Next Steps

✅ Now that our model is approved, head over to the [deployment-pipeline](deployment-pipeline.ipynb) or [batch-pipeline](batch-pipeline.ipynb) notebook to test your model in staging and promote to production.

## Clean up

Execute the following cell to delete any registered models.

In [ ]:
response = sm_client.list_model_packages(ModelPackageGroupName=project_name)
for model_package in response["ModelPackageSummaryList"]:
    print("Deleting Version {}".format(model_package["ModelPackageArn"].split("/")[-1]))
    sm_client.delete_model_package(ModelPackageName=model_package["ModelPackageArn"])

Execute the following cell to delete cloudformation stacks

1. SageMaker Pipeline Workflow Model Package Group

In [ ]:
import boto3

cfn = boto3.client("cloudformation")

for stack_name in [
    f"sagemaker-{project_name}-pipeline",
]:
    print("Deleting stack: {}".format(stack_name))
    cfn.delete_stack(StackName=stack_name)
    cfn.get_waiter("stack_delete_complete").wait(StackName=stack_name)

The following code will clean up all objects in the artifact bucket and delete the SageMaker project.

In [ ]:
s3_resource = boto3.resource("s3")
s3_artifact_bucket = s3_resource.Bucket(artifact_bucket)
s3_artifact_bucket.object_versions.delete()
print("Artifact bucket objects deleted")

sm_client.delete_project(ProjectName=project_name)
print("SageMaker Project deleted")